# Seq2bg

## Imports

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from Seq2Seq import Seq2Seq
from torch.utils.data import DataLoader
import io
import imageio
from ipywidgets import widgets, HBox

In [4]:

import numpy as np

import os
import cv2



# Use GPU if available
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Load data

In [28]:
input_path = "highway/input"
gt_path = "highway/groundtruth/"
dataset = {}


# Llegim totes les imatges d'input i les incorporam a una ED de numpy
imatges_input = os.listdir(input_path)
imatges_input.sort()

input_data = np.zeros((len(imatges_input), 3, 240, 320))


for idx, imatge in enumerate(imatges_input):
    aux = cv2.imread(input_path +"/" + imatge)
    input_data[idx,:,:,:] = np.moveaxis(aux, -1, 0)
    
# Llegim totes les imatges de GT i les incorporam a una ED de numpy
input_gt = np.zeros((len(imatges_input), 1, 240, 320))
imatges_gt = os.listdir(gt_path)
imatges_gt.sort()

for idx, imatge in enumerate(imatges_gt):
    aux = cv2.imread(gt_path +"/" + imatge, cv2.IMREAD_GRAYSCALE)
    input_gt[idx,:,:,:] = aux
    


In [29]:
dataset = zip(input_data, input_gt)



In [ ]:
def collate(batch):

    # Add channel dim, scale pixels between 0 and 1, send to GPU
    batch = torch.tensor(batch).unsqueeze(1)     
    batch = batch / 255.0                        
    batch = batch.to(device)                     

    # Randomly pick 10 frames as input, 11th frame is target
    rand = np.random.randint(10,20)                     
    return batch[:,:,rand-10:rand], batch[:,:,rand]     


# Training Data Loader
train_loader = DataLoader(dataset, shuffle=True, 
                        batch_size=16, collate_fn=collate)

# Validation Data Loader
val_loader = DataLoader(val_data, shuffle=True, 
                        batch_size=16, collate_fn=collate)